<a href="https://colab.research.google.com/github/nautiyal-sarthak/tesla-dash/blob/main/AuraDash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# KM travelled per day/Month

#Max kwh by time
#Max range per time

#TRIP DETAILS
#TOTAL distance traveled per day/MOnth
#trip efficiency per day/Month + temprature
#relation between duration of trip and efficiency
#wh/km across time + temprature

#PHANTOM LOSS
#total phantom loss per day/Month
#pantom loss rate per day/Month + temprature
#phantom loss rate with and without sentry mode across time + temprature

#Money Saved
# total cost comparison between ice and aura per day , month , year
# relation between distance traved and money savings
# relation between temprature and money savings

In [1]:
%%sh
pip install -q dash
pip install -q dash_core_components
pip install -q dash_html_components
pip install -q dash_bootstrap_components
pip install -q dash_table



In [2]:
%%sh
# get ngrok
curl -O https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14.0M  100 14.0M    0     0  5058k      0  0:00:02  0:00:02 --:--:-- 5056k


In [3]:
from google.colab import drive
import pandas as pd

In [5]:
#Reading Data from Google Drive

drive.mount('/gdrive')
trip_df = pd.read_csv("/gdrive/My Drive/aura_data/trip.csv",index_col="trip_id")
charge_df = pd.read_csv("/gdrive/My Drive/aura_data/charge.csv",index_col="session_id")
park_df = pd.read_csv("/gdrive/My Drive/aura_data/phantom.csv",index_col="park_id")


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [10]:
%%writefile dash_app.py

import pathlib
import re
import json
from datetime import datetime
import flask
import dash
import dash_table
import matplotlib.colors as mcolors
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

###################Making DataSets#################
trip_df = pd.read_csv("/gdrive/My Drive/aura_data/trip.csv",index_col="trip_id")
park_df = pd.read_csv("/gdrive/My Drive/aura_data/phantom.csv",index_col="park_id")
charge_df = pd.read_csv("/gdrive/My Drive/aura_data/charge.csv",index_col="session_id")


park_df = park_df[park_df["duration"]/(60*60) > 1]
park_df["avg_temp"] = park_df['avg_temp'].apply(np.ceil)
trip_df["avg_temp"] = trip_df["avg_temp"].apply(np.ceil)

########TRIP###############
trip_df["trip_start_mm"] = pd.to_datetime(trip_df["trip_start_date"]).dt.strftime('%Y-%m')
trip_dis_date = trip_df.groupby("trip_start_date").sum()["distance_traveled"]
trip_eff_temp = trip_df.groupby("avg_temp").mean()["efficiency"]
trip_temp_wh_date = trip_df.groupby("trip_start_mm").mean()[["wh/km","avg_temp","max_kwh","real_dis"]]

#########PARK LOSS############
park_loss_df = park_df.groupby("park_start_date").sum()[["battery_%_lost","duration"]]
park_loss_df["duration"] = park_loss_df["duration"]/(60*60)
park_loss_df = park_loss_df[park_loss_df["duration"] < 24]
park_loss_df.reset_index(inplace=True)
park_loss_df["park_start_mm"]=pd.to_datetime(park_loss_df["park_start_date"]).dt.strftime('%Y-%m')

park_loss_mm_df = park_loss_df.groupby("park_start_mm").mean()[["battery_%_lost","duration"]]
park_loss_sentry_df = park_df[park_df["wh_loss_rate_per_hr"] > 0].groupby(["avg_temp","park_sentry_mode"]).mean()["wh_loss_rate_per_hr"]

############Money Saved#########
# total cost comparison between ice and aura per day , month , year
trip_cost_km_ev = trip_df[["trip_start_date","cost_ev","distance_traveled"]]
trip_cost_km_ev["type"] = "aura"
trip_cost_km_ev.rename(columns={"cost_ev": "cost"},inplace=True)

trip_cost_km_ice = trip_df[["trip_start_date","cost_ice","distance_traveled"]]
trip_cost_km_ice["type"] = "ice"
trip_cost_km_ice.rename(columns={"cost_ice": "cost"},inplace=True)

trip_cost_km = pd.concat([trip_cost_km_ev,trip_cost_km_ice])
trip_cost_km = trip_cost_km.groupby(["trip_start_date","type"]).sum()

# relation between distance traved and money savings
dis_sav = trip_df.groupby("distance_traveled").mean()["money_saved"]
# relation between temprature and money savings
temp_sav = trip_df.groupby("avg_temp").mean()["money_saved"]
###################################################

EXTERNAL_STYLESHEETS = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]
TESLA_LOGO = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTotNbQQhpxWDf5-DhC5vwXo0PMzYynpeFRAQ&usqp=CAU"

NAVBAR = dbc.Navbar(
    children=[
        html.A(
            # Use row and col to control vertical alignment of logo / brand
            dbc.Row(
                [
                    dbc.Col(html.Img(src=TESLA_LOGO, height="30px")),
                    dbc.Col(
                        dbc.NavbarBrand("Aura's Report Card", className="ml-2")
                    ),
                ],
                align="center",
                no_gutters=True,
            ),
            href="https://ts.la/sarthak94017",
        )
    ],
    color="dark",
    dark=True,
    sticky="top",
)


TOT_KM = dbc.Jumbotron(
    [
        html.H6(children="Distance Travelled"),
        html.Hr(className="my-2"),
        html.Label(str(float("{:.2f}".format(trip_df["distance_traveled"].sum()))) + " Km"),
    ]
)

TOT_CAD_SPENT = dbc.Jumbotron(
    [
        html.H6(children="Money Spent"),
        html.Hr(className="my-2"),
        html.Label(str(float("{:.2f}".format(charge_df["cost"].sum()))) + " CAD"),
    ]
)

TOT_CAD_SAVED = dbc.Jumbotron(
    [
        html.H6(children="Money Saved*"),
        html.Hr(className="my-2"),
        html.Label(str(float("{:.2f}".format(trip_df["money_saved"].sum()))) + " CAD"),
    ]
)

AVG_EFF = dbc.Jumbotron(
    [
        html.H6(children="Average Efficiency"),
        html.Hr(className="my-2"),
        html.Label(str(float("{:.2f}".format(trip_df["efficiency"].mean()))) + " %"),
    ]
)

AVG_WH = dbc.Jumbotron(
    [
        html.H6(children="Average Wh/Km"),
        html.Hr(className="my-2"),
        html.Label(float("{:.2f}".format(trip_df["wh/km"].mean()))),
    ]
)

def display_tot_cap():

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])


    # Add traces
    fig.add_trace(
        go.Scatter(x=trip_temp_wh_date.index, y=trip_temp_wh_date["max_kwh"], name="Energy capacity"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=trip_temp_wh_date.index, y=trip_temp_wh_date["real_dis"], name="Distance on full charge (KM)"),
        secondary_y="Secondary" == 'Secondary',
    )

    # Add figure title
    fig.update_layout(
        title_text="Energy Capacity"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(
        title_text="Energy capacity (KWh)", 
        secondary_y=False)
    fig.update_yaxes(
        title_text="Distance on full charge (KM)", 
        secondary_y=True)

    return fig


TOT_KWH_PLOT = [
    dbc.CardHeader(html.H5("Energy Capacity")),
    dbc.CardBody(
        [
          dcc.Graph(id="line-chart-max-kwh",figure = display_tot_cap()),
        ],
        style={"marginTop": 0, "marginBottom": 0},
    ),
]


###### PLOTS for TRIP############
def display_trip_wh():
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=trip_temp_wh_date.index, y=trip_temp_wh_date["wh/km"], name="wh/km"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=trip_temp_wh_date.index, y=trip_temp_wh_date["avg_temp"], name="Temprature"),
        secondary_y="Secondary" == 'Secondary',
    )

    # Add figure title
    fig.update_layout(
        title_text="wh/km across time"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(
        title_text="wh/km", 
        secondary_y=False)
    fig.update_yaxes(
        title_text="Temprature", 
        secondary_y=True)

    return fig

#TOTAL distance traveled per day/MOnth
DIS_TIME = dbc.Card(dcc.Graph(id="km_per_time",figure = px.bar(trip_dis_date, x=trip_dis_date.index, y="distance_traveled",title="Distance Travelled Per Day",labels={
                     "x": "Date",
                     "distance_traveled": "Distance Travelled (km)"
                 })))
#trip efficiency per day/Month + temprature
EFF_TIME_TEMP = dbc.Card(dcc.Graph(id="eff_per_time",figure = px.area(trip_eff_temp, x=trip_eff_temp.index, y="efficiency",title="Relation of Temprature and Efficiency",labels={
                     "x": "Temprature",
                     "efficiency": "Efficiency %"
                 })))
#wh/km across time + temprature
WH_TIME_TEMP = dbc.Card(dcc.Graph(id="wh_per_time",figure = display_trip_wh()))
################################


##############PLOTS for LOSS########
def display_loss_wh():

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])


    # Add traces
    fig.add_trace(
        go.Bar(x=park_loss_mm_df.index, y=park_loss_mm_df["battery_%_lost"], name="Phantom Loss %"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Scatter(x=park_loss_mm_df.index, y=park_loss_mm_df["duration"], name="Parking Duration (hr)"),
        secondary_y="Secondary" == 'Secondary',
    )

    # Add figure title
    fig.update_layout(
        title_text="Phantom Loss Across Time"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Time")

    # Set y-axes titles
    fig.update_yaxes(
        title_text="Phantom Loss %", 
        secondary_y=False)
    fig.update_yaxes(
        title_text="Parking Duration (hr)", 
        secondary_y=True)

    return fig

#total phantom loss per day/Month
LOSS_TIME = dbc.Card(dcc.Graph(id="loss_per_time",figure = display_loss_wh()))
#pantom loss rate per day/Month + temprature
LOSS_TIME_TEMP = dbc.Card(dcc.Graph(id="loss_per_temp",figure = px.bar(park_loss_sentry_df, x=park_loss_sentry_df.index.get_level_values('avg_temp'), 
                                                                       y="wh_loss_rate_per_hr",barmode="group",color=park_loss_sentry_df.index.get_level_values("park_sentry_mode"),title="Phantom Loss and sentry mode"
                                                                       ,labels={
                     "x": "Temprature",
                     "wh_loss_rate_per_hr": "Loss Rate per Hour (Wh)",
                     "color":"Sentry Mode"
                 })))
####################################

##############PLOTS for savings########
# total cost comparison between ice and aura per day , month , year
def display_cost_wh():
    aura = trip_cost_km[trip_cost_km.index.isin(['aura'], level=1)]
    ice = trip_cost_km[trip_cost_km.index.isin(['ice'], level=1)]
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(x=aura.index.get_level_values("trip_start_date"), y=aura["cost"], name="Aura's expense (CAD)"),
        secondary_y=False,
    )

    fig.add_trace(
        go.Bar(x=ice.index.get_level_values("trip_start_date"), y=ice["cost"], name="ICE's expense (CAD)"),
        secondary_y=False,
    )

    fig.update_layout(barmode='group')

    fig.add_trace(
        go.Scatter(x=aura.index.get_level_values("trip_start_date"), y=aura["distance_traveled"], name="Distance Traveled (km)"),
        secondary_y="Secondary" == 'Secondary',
    )

    # Add figure title
    fig.update_layout(
        title_text="Cost Comparison between Aura and ICE"
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Date")

    # Set y-axes titles
    fig.update_yaxes(
        title_text="Cost (CAD)", 
        secondary_y=False)
    fig.update_yaxes(
        title_text="Distance Traveled (km)", 
        secondary_y=True)

    return fig

COST_TIME = dbc.Card(dcc.Graph(id="cost_time",figure = display_cost_wh()))
# relation between distance traved and money savings
DIST_SAV = dbc.Card(dcc.Graph(id="dist_sav",figure = px.scatter(dis_sav, x=dis_sav.index, y="money_saved",title="Relation Between Distance Traveled and Money Spent on Fuel",labels={
                     "x": "Distance Travelled",
                     "money_saved": "Savings (CAD)"
                 })))
# relation between temprature and money savings
TEMP_SAV = dbc.Card(dcc.Graph(id="temp_sav",figure = px.line(temp_sav, x=temp_sav.index, y="money_saved",title="Relation Between Temprature and Money Saved",labels={
                     "x": "Temprature",
                     "money_saved": "Savings (CAD)"
                 })))
####################################

DETAIL_PLOTS = [
    dbc.CardHeader(html.H5("Trip, Parking and Savings Details")),
    dbc.CardBody(
        [
            dbc.Row(
                [
                    dbc.Col(
                        [
                            dcc.Tabs(
                                id="tabs",
                                children=[
                                    dcc.Tab(
                                        label="Trip Details",
                                        children=[
                                            dcc.Loading(
                                                id="loading-treemap",
                                                children=[dbc.Row([dbc.Col(DIS_TIME),
                                                                   dbc.Col(EFF_TIME_TEMP),], style={"marginTop": 30}),
                                                          dbc.Row([dbc.Col(WH_TIME_TEMP),], style={"marginTop": 30}),
                                                          ],
                                                type="default",
                                            )
                                        ],
                                    ),
                                    dcc.Tab(
                                        label="Phantom Loss",
                                        children=[
                                            dcc.Loading(
                                                id="loading-wordcloud",
                                                children=[
                                                    dbc.Row([dbc.Col(LOSS_TIME),], style={"marginTop": 30}),
                                                    dbc.Row([dbc.Col(LOSS_TIME_TEMP),], style={"marginTop": 30}),
                                                ],
                                                type="default",
                                            )
                                        ],
                                    ),                                    dcc.Tab(
                                        label="Savings",
                                        children=[
                                            dcc.Loading(
                                                id="loading-wordcloud1",
                                                children=[
                                                    dbc.Row([dbc.Col(DIST_SAV),dbc.Col(TEMP_SAV),], style={"marginTop": 30}),
                                                    dbc.Row([dbc.Col(COST_TIME),], style={"marginTop": 30}),
                                                ],
                                                type="default",
                                            )
                                        ],
                                    ),
                                ],
                            )
                        ],
                        md=12,
                    ),
                ]
            )
        ]
    ),
]


BODY = dbc.Container(
    [
        dbc.Row([dbc.Col(TOT_KM),dbc.Col(AVG_EFF),dbc.Col(AVG_WH),dbc.Col(TOT_CAD_SPENT),dbc.Col(TOT_CAD_SAVED),], style={"marginTop": 30}),
        dbc.Row([dbc.Col(dbc.Card(TOT_KWH_PLOT)),], style={"marginTop": 30}),
        dbc.Row([dbc.Col(dbc.Card(DETAIL_PLOTS)),], style={"marginTop": 30}),
    ],
)


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.layout = html.Div(children=[NAVBAR, BODY,html.Label("*Assumptions -->fule price = 1.20/L and Fule used per 100km = 7 L")])


if __name__ == '__main__':
    app.run_server(debug=True,dev_tools_hot_reload=False)

Overwriting dash_app.py


In [11]:
# launch ngrok
get_ipython().system_raw('./ngrok http 8050 &')

In [12]:
%%sh
# get url with ngrok
curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://b14dd3a6d876.ngrok.io


In [13]:
!python dash_app.py

dash_app.py:49: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

dash_app.py:53: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "dash_app" (lazy loading)